In [ ]:
!pip install --quiet stanza transformers datasets tokenizers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [ ]:
!unzip -q lessons_final.zip -d ./lessons_final


In [ ]:
pip install --upgrade torch torchvision torchaudio


  Using cached torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached torchvision-0.22.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached torchaudio-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.6 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using ca

In [ ]:
pip install transformers==4.28.1


In [ ]:
# !pip install --quiet stanza transformers datasets tokenizers torch

import os, pickle, random
import torch
from torch import nn
torch.autograd.set_detect_anomaly(True)
from datasets import Dataset
from transformers import (
    GPT2Config, GPT2TokenizerFast,
    Trainer, TrainingArguments, GPT2LMHeadModel
)
from sklearn.model_selection import train_test_split

# STEP 1 – Setup
import stanza
stanza.download("en", verbose=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MAX_COMBOS = 1024
combo2id = {"<UNK>": 0}
_next_id = 1
def dynamic_get_combo_id(pos, dep, ner, feats):
    global _next_id
    key = (pos, dep, ner, feats)
    if key not in combo2id and _next_id < MAX_COMBOS:
        combo2id[key] = _next_id
        _next_id += 1
    return combo2id.get(key, 0)

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

import re

# STEP 2 – Load pre-parsed lessons
pkl_folder = "./lessons_final/lessons_final"
lessons = {}

def extract_vol_lesson(filename):
    match = re.match(r'v(\d+)_l(\d+)\.pkl', filename)
    return (int(match.group(1)), int(match.group(2))) if match else (999, 999)

all_files = os.listdir(pkl_folder)
pkl_files = sorted(
    [f for f in all_files if re.match(r'v\d+_l\d+\.pkl', f)],
    key=extract_vol_lesson
)

for fname in pkl_files:
    with open(os.path.join(pkl_folder, fname), "rb") as f:
        lessons[fname] = pickle.load(f)
    print(f"✅ Loaded {fname} ({len(lessons[fname])} docs)")

def filter_short_docs(docs, min_tokens=2):
    kept = []
    for i, d in enumerate(docs):
        ids, _ = encode_doc(d)
        if len(ids) >= min_tokens:
            kept.append(d)
        else:
            print(f"⚠️ Skipping doc #{i} in this lesson: only {len(ids)} tokens")
    return kept


# STEP 3 – Combo extraction and encoding
def scan_doc(doc):
    ents = [(e.start_char, e.end_char, e.type) for e in doc.ents]
    for w in [w for s in doc.sentences for w in s.words]:
        ner = next((t for s0, e0, t in ents if s0 <= w.start_char < e0), "O")
        dynamic_get_combo_id(w.upos, w.deprel, ner, w.feats)

def encode_doc(doc):
    text = doc.text
    enc = tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
    ents = [(e.start_char, e.end_char, e.type) for e in doc.ents]
    words = [w for s in doc.sentences for w in s.words]

    combo_ids = []
    for start, end in enc["offset_mapping"]:
        if start == end or end <= start:
            combo_ids.append(0)
            continue

        match = None
        for w in words:
            # allow small overlap fuzziness (fixes edge mismatches)
            if not (w.end_char <= start or w.start_char >= end):
                match = w
                break

        if match:
            ner = next((t for s0, e0, t in ents if s0 <= match.start_char < e0), "O")
            combo_ids.append(dynamic_get_combo_id(match.upos, match.deprel, ner, match.feats))
        else:
            combo_ids.append(0)
    num_total, num_missed = 0, 0
    num_total += len(enc["offset_mapping"])
    num_missed += sum(1 for c in combo_ids if c == 0)

    # print(f"💡 Miss rate: {100 * num_missed / num_total:.2f}%")

    return enc["input_ids"], combo_ids

def chunkify_docs(docs, max_len=64):
    """Each doc becomes one padded (or truncated) sample."""
    samples = []
    for doc in docs:
        ids, cmb = encode_doc(doc)
        ids = ids[:max_len]
        cmb = cmb[:max_len]
        pad = max_len - len(ids)
        if pad:
            ids += [tokenizer.pad_token_id] * pad
            cmb += [0] * pad
        samples.append({"input_ids": ids, "combo_ids": cmb})
    return Dataset.from_list(samples)

def collate(batch, max_len=64):
    ids, cmb = [], []
    for b in batch:
        x = b["input_ids"][:max_len]
        y = b["combo_ids"][:max_len]
        pad = max_len - len(x)
        x += [tokenizer.pad_token_id] * pad
        y += [0] * pad
        ids.append(x)
        cmb.append(y)

    # Create CPU tensors (no device=)
    ids = torch.tensor(ids)
    cmb = torch.tensor(cmb)

    # Build attention mask and labels
    mask = (ids != tokenizer.pad_token_id).long()
    labels = ids.clone()
    labels[ids == tokenizer.pad_token_id] = -100

    # If ALL masked, unmask [0,0] so loss != nan
    if (labels == -100).all():
        labels[0, 0] = ids[0, 0]
        print("⚠️ All labels masked—unmasking [0,0] to avoid nan loss")

    return {
        "input_ids":      ids,
        "combo_ids":      cmb,
        "attention_mask": mask,
        "labels":         labels,
    }

class SyntaxGPT(GPT2LMHeadModel):
    def __init__(self, cfg):
        super().__init__(cfg)
        H = cfg.n_embd
        self.combo_embed = nn.Embedding(MAX_COMBOS, H)
        self.merge = nn.Linear(2 * H, H, bias=False)
        self.stored_combo_ids = None
        self.nums = 0

    def prepare_inputs_for_generation(self, input_ids, past_key_values=None, attention_mask=None, **kwargs):
        if past_key_values is None:
            combo_ids = self.stored_combo_ids.to(input_ids.device)
        else:
            combo_ids = input_ids.new_zeros(input_ids.shape)
            attention_mask = None
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "past_key_values": past_key_values,
            "combo_ids": combo_ids,
        }

    def forward(self, input_ids=None, combo_ids=None, attention_mask=None,
                labels=None, inputs_embeds=None, past_key_values=None, **kwargs):
        # print(f"hello {self.nums}")
        # self.nums += 1

        if inputs_embeds is None:
            if combo_ids is None or combo_ids.size(1) != input_ids.size(1):
                raise ValueError("combo_ids must align with input_ids")
            tok = self.transformer.wte(input_ids)
            cmb = self.combo_embed(combo_ids.to(tok.device))
            inputs_embeds = self.merge(torch.cat([tok, cmb], dim=-1))

        return GPT2LMHeadModel.forward(
            self,
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            labels=labels,
            past_key_values=past_key_values,
            **kwargs
        )

cfg = GPT2Config.from_pretrained("gpt2", vocab_size=tokenizer.vocab_size)
model = SyntaxGPT(cfg).to(device)
model.config.pad_token_id = tokenizer.pad_token_id

from collections import defaultdict
from sklearn.model_selection import train_test_split

combined_val_docs = []
base_lr = 5e-4
n_epochs = 10
TOTAL = len(lessons)

# Helper to build a HuggingFace Dataset from precomputed encodings
from datasets import Dataset
def build_dataset(preprocessed, max_len=64):
    samples = []
    for ids, combo_ids in preprocessed:
        ids = ids[:max_len]
        combo = combo_ids[:max_len]
        pad = max_len - len(ids)
        if pad:
            ids   = ids   + [tokenizer.pad_token_id] * pad
            combo = combo + [0] * pad
        samples.append({"input_ids": ids, "combo_ids": combo})
    return Dataset.from_list(samples)

# ─────────────────────────────────────────────────────────────
# Now the main loop
# ─────────────────────────────────────────────────────────────
for idx, (lesson_name, raw_docs) in enumerate(lessons.items(), 1):
    # 1) Precompute encodings once
    preprocessed = []
    for d in raw_docs:
        ids, combos = encode_doc(d)   # only call encode_doc here
        preprocessed.append((ids, combos))

    # 2) Filter short docs (<2 tokens)
    filtered = []
    for i, (ids, combos) in enumerate(preprocessed):
        if len(ids) >= 2:
            filtered.append((ids, combos))
        else:
            print(f"⚠️ Skipping doc #{i} in {lesson_name}: only {len(ids)} tokens")

    if len(filtered) < 2:
        print(f"⚠️ Lesson {lesson_name} has too few valid docs ({len(filtered)}), skipping")
        continue

    # 3) Split into train/val on the filtered, precomputed list
    train_items, val_items = train_test_split(filtered, test_size=0.1, random_state=42)
    combined_val_docs += [combos for _, combos in val_items]  # for final evaluation

    print(f"\n▶ Lesson {idx}/{TOTAL}: {lesson_name} | "
          f"{len(train_items)} train | {len(val_items)} val")

    # 4) Build Datasets directly, no re-encoding
    train_set = build_dataset(train_items, max_len=64)
    val_set   = build_dataset(val_items,   max_len=64)
    if len(val_set) >= 2:
        eval_ds = val_set
        eval_strategy = "epoch"
    else:
        print(f"⚠️ Skipping eval for {lesson_name} (only {len(val_set)} val samples)")
        eval_ds = None
        eval_strategy = "no"
    args = TrainingArguments(
        output_dir=f"ckpt_{lesson_name}",
        num_train_epochs=n_epochs,
        learning_rate=base_lr,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy=eval_strategy,
        logging_strategy="epoch",
        logging_steps=n_epochs // 5,
        save_strategy="no",
        report_to="none",
        fp16=False,
        remove_unused_columns=False
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_set,
        eval_dataset=val_set if len(val_set)>=2 else None,
        data_collator=collate,  # or your existing collate
    )
    trainer.train()

✅ Loaded v1_l1.pkl (8 docs)
✅ Loaded v1_l2.pkl (8 docs)
✅ Loaded v1_l3.pkl (13 docs)
✅ Loaded v1_l4.pkl (12 docs)
✅ Loaded v1_l5.pkl (14 docs)
✅ Loaded v1_l6.pkl (48 docs)
✅ Loaded v1_l7.pkl (14 docs)
✅ Loaded v1_l8.pkl (25 docs)
✅ Loaded v1_l9.pkl (9 docs)
✅ Loaded v1_l10.pkl (21 docs)
✅ Loaded v1_l11.pkl (13 docs)
✅ Loaded v1_l12.pkl (18 docs)
✅ Loaded v1_l13.pkl (11 docs)
✅ Loaded v1_l14.pkl (25 docs)
✅ Loaded v1_l15.pkl (16 docs)
✅ Loaded v1_l16.pkl (9 docs)
✅ Loaded v1_l17.pkl (27 docs)
✅ Loaded v1_l18.pkl (18 docs)
✅ Loaded v1_l19.pkl (11 docs)
✅ Loaded v1_l20.pkl (31 docs)
✅ Loaded v1_l21.pkl (9 docs)
✅ Loaded v1_l22.pkl (40 docs)
✅ Loaded v1_l23.pkl (8 docs)
✅ Loaded v1_l24.pkl (28 docs)
✅ Loaded v1_l25.pkl (42 docs)
✅ Loaded v1_l26.pkl (18 docs)
✅ Loaded v1_l27.pkl (24 docs)
✅ Loaded v1_l28.pkl (8 docs)
✅ Loaded v1_l29.pkl (21 docs)
✅ Loaded v1_l30.pkl (15 docs)
✅ Loaded v1_l31.pkl (34 docs)
✅ Loaded v1_l32.pkl (11 docs)
✅ Loaded v1_l33.pkl (17 docs)
✅ Loaded v1_l34.pkl (16 do

Step,Training Loss
1,10.880100
2,2.742000
3,2.087200
4,1.178500
5,0.999300
6,0.764500
7,0.711600
8,0.630100
9,0.560300
10,0.541500



▶ Lesson 2/345: v1_l2.pkl | 7 train | 1 val
⚠️ Skipping eval for v1_l2.pkl (only 1 val samples)


Step,Training Loss
1,0.746800
2,1.326300
3,1.641600
4,1.297300
5,0.956500
6,0.753100
7,0.613200
8,0.520300
9,0.466300
10,0.429000



▶ Lesson 3/345: v1_l3.pkl | 11 train | 2 val


...


▶ Lesson 344/345: v4_l45.pkl | 28 train | 4 val


Epoch,Training Loss,Validation Loss
1,7.395300,7.475904
2,6.343600,7.164868
3,5.662300,6.900085
4,5.079900,6.804727
5,4.750400,6.770178
6,4.458700,6.776848
7,4.234600,6.759943
8,4.001500,6.737232
9,3.885100,6.733875
10,3.785000,6.735386


⚠️ Skipping doc #0 in v4_l46.pkl: only 1 tokens

▶ Lesson 345/345: v4_l46.pkl | 62 train | 7 val


Epoch,Training Loss,Validation Loss
1,7.240200,6.573377
2,5.803100,6.382441
3,4.967100,6.067444
4,4.337700,5.903954
5,3.868800,5.876849
6,3.450400,5.835340
7,3.177300,5.794540
8,2.971200,5.812356
9,2.808600,5.836004
10,2.757700,5.851102


ValueError: too many values to unpack (expected 2)

In [ ]:
# Assuming `model` and `tokenizer` are already loaded/fine-tuned
save_dir = "my_syntax_gpt_model"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


('my_syntax_gpt_model/tokenizer_config.json',
 'my_syntax_gpt_model/special_tokens_map.json',
 'my_syntax_gpt_model/vocab.json',
 'my_syntax_gpt_model/merges.txt',
 'my_syntax_gpt_model/added_tokens.json',
 'my_syntax_gpt_model/tokenizer.json')

In [ ]:
save_dir = "my_syntax_gpt_model"

model = SyntaxGPT.from_pretrained(save_dir).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(save_dir)


In [ ]:
# 🔥 new imports at the top
from torch.nn.utils import clip_grad_norm_
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# ─────────────────────────────────────────────────────────────
# load TREC and Stanza‑parsed docs
# ─────────────────────────────────────────────────────────────
trec = load_dataset("trec")
label_map = trec["train"].features["coarse_label"].names

with open("trec_train_docs.pkl", "rb") as f:
    parsed_train_docs = pickle.load(f)
with open("trec_test_docs.pkl", "rb") as f:
    parsed_test_docs = pickle.load(f)

assert len(parsed_train_docs) == len(trec["train"])
assert len(parsed_test_docs)  == len(trec["test"])

# ─────────────────────────────────────────────────────────────
# encode_doc_with_label: builds input_ids, combo_ids, labels
# ─────────────────────────────────────────────────────────────
def encode_doc_with_label(doc, label_text, max_len=64):
    # prompt tokens
    prompt_text = doc.text.strip()
    prompt_ids  = tokenizer(prompt_text, add_special_tokens=False)["input_ids"]
    # label tokens
    label_ids   = tokenizer.encode(label_text.strip(), add_special_tokens=False)

    # full input = prompt + label
    full_input_ids = prompt_ids + label_ids
    # mask prompt for loss, keep label tokens
    labels = [-100]*len(prompt_ids) + label_ids

    # get combo_ids for prompt only, pad to full length
    _, combo_ids = encode_doc(doc)
    combo_ids = combo_ids[:len(full_input_ids)]
    combo_ids += [0]*(len(full_input_ids) - len(combo_ids))

    # truncate/pad to max_len
    full_input_ids = full_input_ids[:max_len]
    labels         = labels[:max_len]
    combo_ids      = combo_ids[:max_len]

    pad = max_len - len(full_input_ids)
    full_input_ids += [tokenizer.pad_token_id]*pad
    labels         += [-100]*pad
    combo_ids      += [0]*pad

    return {
        "input_ids": full_input_ids,
        "combo_ids": combo_ids,
        "labels":    labels,
    }

# ─────────────────────────────────────────────────────────────
# dataset wrapper
# ─────────────────────────────────────────────────────────────
class TrecSyntaxDataset(Dataset):
    def __init__(self, parsed_docs, label_ids, max_len=64):
        self.samples = []
        for doc, label_id in tqdm(zip(parsed_docs, label_ids), total=len(parsed_docs)):
            sample = encode_doc_with_label(doc, label_map[label_id], max_len)
            self.samples.append(sample)

    def __getitem__(self, idx):
        return self.samples[idx]

    def __len__(self):
        return len(self.samples)

# ─────────────────────────────────────────────────────────────
# build datasets & collator
# ─────────────────────────────────────────────────────────────
train_set = TrecSyntaxDataset(parsed_train_docs, trec["train"]["coarse_label"])
val_set   = TrecSyntaxDataset(parsed_test_docs,  trec["test"]["coarse_label"])

# 🚨 Modify collate_trec: protect against fully masked labels
def collate_trec(batch):
    ids   = torch.tensor([b["input_ids"] for b in batch])
    cmb   = torch.tensor([b["combo_ids"] for b in batch])
    labs  = torch.tensor([b["labels"]    for b in batch])
    mask  = (ids != tokenizer.pad_token_id).long()

    # Protect: if all labels masked (-100), unmask one
    for i in range(labs.shape[0]):
        if (labs[i] != -100).sum() == 0:
            labs[i, -1] = ids[i, -1]  # unmask last token

    return {
        "input_ids":      ids,
        "combo_ids":      cmb,
        "attention_mask": mask,
        "labels":         labs,
    }

# 🚨 Slightly change Trainer setup
args = TrainingArguments(
    output_dir="syntaxgpt_trec_ckpt",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,   # 📈 allow up to 10 epochs
    learning_rate=5e-4,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    # save_strategy="no",
    save_strategy="epoch",
    report_to="none",
    fp16=False,                    # 🚫 Turn off fp16 for stability
    gradient_accumulation_steps=1,
    # gradient_clip_val=1.0,          # 🚀 Clip large gradients (Trainer accepts it now)
    max_grad_norm=1.0,
    remove_unused_columns=False,
    load_best_model_at_end=True,    # 🚨 enable early stopping
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# ✅ add compute_metrics again
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    labels = np.array(labels)

    # Only count labels that were actually predicted
    valid_idx = labels != -100
    preds = preds[valid_idx]
    labels = labels[valid_idx]

    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_set,
    eval_dataset=val_set,
    data_collator=collate_trec,
    compute_metrics=compute_metrics,
)

trainer.train()

100%|██████████| 500/500 [00:00<00:00, 2080.01it/s]
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.904700,0.797544,0.323821,0.105966,0.259806,0.147665
2,0.822700,0.828094,0.308933,0.097289,0.244666,0.138072
3,0.765400,0.896108,0.320099,0.102776,0.255868,0.144932
4,0.712900,0.980980,0.307692,0.097610,0.243465,0.137988
5,0.649800,0.813717,0.323821,0.100817,0.261501,0.144737
6,0.589200,0.805879,0.320099,0.103184,0.259870,0.145891
7,0.562300,0.956926,0.312655,0.098099,0.249281,0.139919
8,0.544400,0.908341,0.320099,0.101480,0.257564,0.144106
9,0.497300,0.845154,0.320099,0.099761,0.258174,0.143047
10,0.464800,0.861960,0.326303,0.102859,0.261662,0.146377


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [ ]:
# 10 pretraining epochs
from sklearn.metrics import precision_recall_fscore_support
import torch.nn.functional as F

correct = 0
total = len(val_set)
all_preds = []
all_golds = []
losses = []

def predict_label(sample):
    prompt_len = next(i for i, x in enumerate(sample["labels"]) if x != -100)

    input_ids = torch.tensor([sample["input_ids"][:prompt_len]]).to(device)
    combo_ids = torch.tensor([sample["combo_ids"][:prompt_len]]).to(device)

    # 📢 Extend combo_ids to match expected beam size
    while combo_ids.size(1) < input_ids.size(1):
        # pad extra 0s at the end
        combo_ids = torch.cat([combo_ids, combo_ids.new_zeros(1,1)], dim=1)

    model.stored_combo_ids = combo_ids

    print(f"prompt: {tokenizer.decode(input_ids[0], skip_special_tokens=True).strip()}")
    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            attention_mask=(input_ids != tokenizer.pad_token_id).long(),
            max_new_tokens=4,
            num_beams=1,  # 🚨 important: disable beam search if possible
        )[0].tolist()

    new_ids = out[prompt_len:]
    return tokenizer.decode(new_ids, skip_special_tokens=True).strip().upper()


label_name_to_id = {name.upper(): i for i, name in enumerate(label_map)}

for sample, gold_idx in zip(val_set, trec["test"]["coarse_label"]):
    pred = predict_label(sample)
    true = label_map[gold_idx].upper()

    print(f"prompt: {tokenizer.decode(sample['input_ids'], skip_special_tokens=True).strip()}")
    print(f" predict: {pred}\n")

    if pred.startswith(true):  # ✅ soft match
        correct += 1
        pred = true  # overwrite pred to ensure consistency in metrics

    all_preds.append(pred)
    all_golds.append(true)

    # Optional: token-level loss
    try:
        input_ids = torch.tensor([sample["input_ids"]]).to(device)
        labels = torch.tensor([sample["labels"]]).to(device)
        attn_mask = (input_ids != tokenizer.pad_token_id).long()
        model.stored_combo_ids = torch.tensor([sample["combo_ids"]]).to(device)

        with torch.no_grad():
            output = model(input_ids=input_ids, attention_mask=attn_mask, labels=labels)
            losses.append(output.loss.item())
    except Exception:
        pass

# 🎯 Accuracy
acc = correct / total

# 🎯 Classification Metrics
gold_ids = [label_name_to_id.get(label, -1) for label in all_golds]
pred_ids = [label_name_to_id.get(p, -1) for p in all_preds]

filtered_gold_ids = []
filtered_pred_ids = []
for g, p in zip(gold_ids, pred_ids):
    if g != -1 and p != -1:
        filtered_gold_ids.append(g)
        filtered_pred_ids.append(p)

p, r, f1, _ = precision_recall_fscore_support(filtered_gold_ids, filtered_pred_ids, average="macro")

# 📢 Final Results
print(f"\n🔍 Evaluation Results (max_new_tokens=4 + startswith logic):")
print(f"Accuracy:  {acc:.4f}")
print(f"Avg Loss:  {sum(losses)/len(losses):.4f}" if losses else "Loss: (not computed)")
print(f"Recall:    {r:.4f} ignore")
print(f"F1 Score:  {f1:.4f} ignore")


prompt: Here is a question: How far is it from Denver to Aspen? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:
prompt: Here is a question: How far is it from Denver to Aspen? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:NUM
 predict: NUMCCC

prompt: Here is a question: What county is Modesto, California in? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:
prompt: Here is a question: What county is Modesto, California in? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:LOC
 predict: LOCYYY

prompt: Here is a question: Who was Galileo? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:
prompt: Here is a question: Who was Galileo? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:HUM
 predict: HUMUMUM

prompt: Here is a question: What is an atom? What is the best label among DESC, ENTY, ABBR, HUM, LOC, NUM? Answer:
prompt: Here is a question: What is an atom? What